<a href="https://colab.research.google.com/github/abel-bernabeu/autoencoder/blob/master/compressor_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compression models training

This notebook is used for training the compression models from the autoencoder.models package.

It is divided in sections, each of them corresponding to an experiment. In each of those experiments we craft an incrementally functional prototype. A series of five experiments leads to the final model.

For each experiment we do:

- Specify the hyperparmeters (hparams).
- Instantiate a model from the autoencoder.models package.
- Create dataloaders with the input patch size and batch size especified in hparams.
- Embed a TensorBoard for visualiazing the training.
- Kick a training session lasting until the number of epochs especified in hparams is reached.

# Setup

Firstly, we download from DropBox the dataset and all the available pre-trained models.

In [ ]:
# When on Google Colab force PyTorch version to 1.4.0 (for compatibility of the .pt files)
try:
    from google.colab import drive
    !pip install torch==1.4.0 torchvision==0.5.0
except:
    pass

# Get the dataset if needed
import os.path
if not os.path.isdir('./data'):
    !rm  -rf image_dataset_part-a.zip
    !mkdir data -p
    !cd data ; wget https://www.dropbox.com/s/91rpg5dqkhhhkzu/image_dataset_part-a.zip
    !cd data ; unzip -q image_dataset_part-a.zip && rm image_dataset_part-a.zip 

# Get the latest source code if needed
if not os.path.isdir('./autoencoder'):
    !wget https://www.dropbox.com/s/jfu0ksttohnklkq/autoencoder-master.zip && \
    unzip -q autoencoder-master.zip && rm autoencoder-master.zip && \
    mv autoencoder-master/autoencoder/ . && \
    rm autoencoder-master -rf

if not os.path.isdir('./share'):
    try:
        # Try to mount share from Google Drive when on Collab
        from google.colab import driveX
        drive.mount('/content/drive/')
        !ln -s  /content/drive/My\ Drive/archive/2020/aidl/ share
    except:
        # The fallback for when not in Collab is to download share from Dropbox
        !wget https://www.dropbox.com/s/76w9gsga8mz5ve4/share.tgz && tar xzf share.tgz && rm share.tgz

In [ ]:
import autoencoder.models
import autoencoder.utils

# Experiment 1: sparsity at 1/2

Our baseline model effort focuses on training the proposed neural network with the maximum possible accuracy, but not investing any effort in quantization of the features.

This model assumes the input is provided in single-precision float values (using 32 bits per pixel color component) and provides a 50% dimensionality reduction of the input (or 1/2 sparsity). The dimensionality reduction is achieved by using 96 channels in the features tensor (as opposed to 192 channels that would be needed if we wanted to keep the dimensionality from the input).

The purpose of this first experiment is to:

- empirically prove that the input can be reconstructed accurately with the kind of neural network that is proposed in the paper.

- set an upper bound on accuracy (which turned out be **43 db**)

- give an estimation of how long it takes to train a state of the art compression model (which turned out to take **4 days on a Tesla P100**)


## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 32,
    'lr' : 1e-3,
    'device' : 'cuda',
    'block_width' : 128,
    'block_height' : 128,
    'hidden_state_num_channels' : 96,
    'train_dataset_size' : 5000,
    'test_dataset_size' : 500,
    'num_epochs' : 12577,
    'num_workers' : 4,
    'name' : "experiment1",
    'port' : 6100,
    'checkpointing_freq' : 10,
    'inference_freq' : 200,
}

!mkdir -p share/{hparams['name']}

## Model instantiation

In [ ]:
model = autoencoder.models.TwitterCompressor(hidden_state_num_channels = hparams['hidden_state_num_channels'])

## Data loaders

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=model, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

# Experiment 2: sparsity at 1/4

In this second experiment we further squeeze the features tensor, going from from 96 channels to only 48, for achieving a 25% dimensionality reduction. Again no quantization is provided.

## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-6,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 48,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 16000,
    'num_workers' : 4,
    'name' : "experiment2",
    'port' : 6200,
    'checkpointing_freq' : 10,
    'inference_freq' : 200,
}

!mkdir -p share/{hparams['name']}

## Model instantiation

In [ ]:
model = autoencoder.models.TwitterCompressor(hidden_state_num_channels = hparams['hidden_state_num_channels'])

## Data loaders

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=model, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

#  Experiment 3: 3 bits quantization

For this third iteration we introduce 3 bits quantization of the features. We transfer the learned weigths from the previous experiment (a model with the same number of feature channels but no quantization).

Then we train the model freezing the encoder, but adjusting the encoder for learning to "undo" the quantization.

## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-6,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 48,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 2500,
    'num_workers' : 4,
    'name' : "experiment3",
    'port' : 6300,
    'checkpointing_freq' : 10,
    'inference_freq' : 200,
}

!mkdir -p  share/{hparams['name']}

## Model instantiation

In [ ]:
qmodel = autoencoder.models.QuantizingCompressionAutoencoder(num_bits=3)

# Transfer learning from the non-quantized model
qmodel.encoder = model.encoder
qmodel.decoder = model.decoder

# Freeze the encoder
for param in qmodel.encoder.parameters():
    param.requires_grad = False

## Data loaders

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=qmodel, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

# Experiment 4: sparsity at 1/8

In this experiment we reduce dimensionality to 1/8, but we do not perform quantization.

## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-6,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 24,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 110000,
    'num_workers' : 4,
    'name' : "experiment4",
    'port' : 6400,
    'checkpointing_freq' : 10,
    'inference_freq' : 200,
}

!mkdir -p share/{hparams['name']}

## Model instantiation

In [ ]:
model = autoencoder.models.TwitterCompressor(hidden_state_num_channels = hparams['hidden_state_num_channels'])

## Data loaders

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=model, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

#  Experiment 5: 6 bits quantization

We transfer the learned encoder and decoder from experiment 4, freeze the encoder weights and further train the decoder for undoing the quantization.



## Hyperparameters

In [ ]:
hparams = {
    'batch_size' : 40,
    'lr' : 1e-10,
    'device' : 'cuda',
    'block_width' : 224,
    'block_height' : 224,
    'hidden_state_num_channels' : 24,
    'train_dataset_size' : 1000,
    'test_dataset_size' : 500,
    'num_epochs' : 20000,
    'num_workers' : 4,
    'name' : "experiment5",
    'port' : 6500,
    'checkpointing_freq' : 10,
    'inference_freq' : 200,
}

!mkdir -p share/{hparams['name']}

## Model instantiation

In [ ]:
qmodel = autoencoder.models.QuantizingCompressionAutoencoder(num_bits=6)

# Transfer learning from the non-quantized model
qmodel.encoder = model.encoder
qmodel.decoder = model.decoder

# Freeze the encoder
for param in qmodel.encoder.parameters():
    param.requires_grad = False

## Data loaders

In [ ]:
train_loader, test_loader, few_train_x, few_train_y, few_test_x, few_test_y = autoencoder.utils.create_dataloaders(hparams)

## TensorBoard

In [ ]:
try:
    # When on Google Colab try to launch an embedded TensorBoard
    from google.colab import drive
    %load_ext tensorboard
    from tensorboard import notebook
    notebook.start('--logdir share/' + hparams['name'] + '/runs/ --port ' + str(hparams['port']))
except:
    pass

## Training

In [ ]:
try:
  autoencoder.utils.train(hparams=hparams, \
        model=qmodel, \
        train_loader=train_loader, \
        test_loader=test_loader, \
        few_train_x=few_train_x, few_train_y=few_train_y, \
        few_test_x=few_test_x, few_test_y=few_test_y)
except KeyboardInterrupt:
    print('Exiting from training early')

# Results

The model from experiment 5 achieves a 10.6 compression ratio with a 39.1 dB PSNR, being the best choice.